## Detección y conteo de fauna africana mediante imágenes aéreas
#### Grupo #2

> NOTEBOOK SOLO DE VERIFICACIÓN: entrenamiento del modelo - Yolov11m

### RECURSO COMPUTACIONAL (inventario rápido del entorno)

In [ ]:
import os, platform, psutil, torch
print(f"SO           : {platform.system()} {platform.release()} ({platform.platform()})")
print(f"CPU          : {psutil.cpu_count(logical=True)} hilos")
print(f"RAM          : {round(psutil.virtual_memory().total/1024**3,2)} GB")
print(f"CUDA         : {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Nombre   : {torch.cuda.get_device_name(0)}")
    print(f"GPU Memoria  : {round(torch.cuda.get_device_properties(0).total_memory/1024**3,2)} GB")
print(f"PyTorch      : {torch.__version__}")

SO           : Windows 10 (Windows-10-10.0.26100-SP0)
CPU          : 24 hilos
RAM          : 31.43 GB
CUDA         : True
GPU Nombre   : NVIDIA GeForce RTX 5080 Laptop GPU
GPU Memoria  : 15.92 GB
PyTorch      : 2.8.0+cu128


### 1) Conexión y configuración del entorno

In [ ]:
from pathlib import Path
import sys, subprocess, importlib

BASE_DIR = Path.cwd()        # Fauna_Detection/
DATA_DIR = BASE_DIR / "DataG2"  # Dataset completo local
assert DATA_DIR.exists(), f"No existe {DATA_DIR}.dataset en 'Fauna_Detection/DataG2'"

print("BASE_DIR:", BASE_DIR)
print("DATA_DIR:", DATA_DIR)

def pip_install(pkg):
    try:
        importlib.import_module(pkg.split("==")[0])
    except:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg], check=True)

# Versiones probadas recientes
pip_install("ultralytics>=8.2.103")
pip_install("opencv-python")
pip_install("pandas")
pip_install("numpy")
pip_install("tqdm")
pip_install("pyyaml")

BASE_DIR: C:\Users\durle\anaconda3\Fauna_Detection
DATA_DIR: C:\Users\durle\anaconda3\Fauna_Detection\DataG2


### 2) Fijación de semilla

In [ ]:
import random, os, numpy as np, torch

class FijadorSemillaYOLO:
    def __init__(self, seed=9292):
        self.seed = seed
    def aplicar(self):
        random.seed(self.seed)
        np.random.seed(self.seed)
        os.environ["PYTHONHASHSEED"] = str(self.seed)
        torch.manual_seed(self.seed)
        torch.cuda.manual_seed_all(self.seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        print(f"Semilla fijada: {self.seed}")

fijador = FijadorSemillaYOLO(9292)
fijador.aplicar()

Semilla fijada: 9292


### 3) Parches
- NO generamos parches; los cargamos directamente.
- Usa los parches existentes en DataG2/train_patches y DataG2/val_patches.
- Mantiene compatibilidad si en algún momento se quisiera regenerar.

In [ ]:
import cv2
import pandas as pd
from tqdm import tqdm

PATCH_SIZE_DEFAULT = 512  # sólo para fallback al normalizar, se leerán tamaños reales con OpenCV

def _listar_imagenes(directorio: Path):
    return [p for p in directorio.glob("*.*") if p.suffix.lower() in [".jpg",".jpeg",".png",".tif",".tiff"]]

# Directorios esperados por el usuario (ya existen con parches)
TRAIN_PATCHES_DIR = DATA_DIR/"images"/"train_patches"
VAL_PATCHES_DIR   = DATA_DIR/"images"/"val_patches"

TEST_DIR  = DATA_DIR/"images"/"test"  # se usará 'test' (imágenes completas) para predicción

assert TRAIN_PATCHES_DIR.exists(), f"Falta {TRAIN_PATCHES_DIR}"
assert VAL_PATCHES_DIR.exists(),   f"Falta {VAL_PATCHES_DIR}"

# CSVs esperados: train_patches.csv y gt.csv (para validación)
TRAIN_PATCHES_CSV = DATA_DIR/"train_patches.csv"
VAL_PATCHES_CSV   = DATA_DIR/"gt.csv" if (DATA_DIR/"gt.csv").exists() else DATA_DIR/"val_patches.csv"

assert TRAIN_PATCHES_CSV.exists(), f"Falta {TRAIN_PATCHES_CSV}"
assert VAL_PATCHES_CSV.exists(),   f"Falta {VAL_PATCHES_CSV}"

print("CSV parches (train):", TRAIN_PATCHES_CSV.name)
print("CSV parches (val)  :", VAL_PATCHES_CSV.name)

CSV parches (train): train_patches.csv
CSV parches (val)  : gt.csv


### 3.1) Conversión de CSV --> etiquetas YOLO .txt (parches)
- Los CSV tienen puntos (x,y) por parche, por clase 'labels'.
- YOLO requiere boxes: class cx cy w h (normalizados).
- Criterio: construir cajas cuadradas alrededor del punto.
- Parámetro BOX_SIDE_PX ajustable (valor conservador 32px).

In [ ]:
from collections import defaultdict

BOX_SIDE_PX = 32  # tamaño de caja alrededor del punto (ancho=alto)

LABELS_ROOT = DATA_DIR / "labels"  # Estructura estandar Ultralytics: labels/<split>
(LABELS_ROOT/"train_patches").mkdir(parents=True, exist_ok=True)
(LABELS_ROOT/"val_patches").mkdir(parents=True, exist_ok=True)

def _inferir_indexado_cero_o_uno(series_labels, n_clases):
    """Si los labels vienen 1..N en vez de 0..N-1, se pasan a 0..N-1 automáticamente."""
    minimo = int(pd.to_numeric(series_labels, errors="coerce").min())
    maximo = int(pd.to_numeric(series_labels, errors="coerce").max())
    if minimo >= 1 and maximo <= n_clases:
        return -1  # restar 1
    return 0      # ya 0-index

### 4) Construcción de dataloaders (adaptado a YOLO)
Se asegura que apunten a los parches existentes y a sus .txt correspondientes

In [ ]:
from dataclasses import dataclass

@dataclass
class YOLODatasetsBuilder:
    train_csv: str
    train_root: str
    val_csv: str
    val_root: str

    def build(self):
        return (Path(self.train_root), Path(self.val_root))

train_dataset, val_dataset = YOLODatasetsBuilder(
    train_csv=str(TRAIN_PATCHES_CSV),
    train_root=str(TRAIN_PATCHES_DIR),
    val_csv=str(VAL_PATCHES_CSV),
    val_root=str(VAL_PATCHES_DIR)
).build()

print("Rutas dataset YOLO:")
print("  train:", train_dataset)
print("  val  :", val_dataset)

Rutas dataset YOLO:
  train: C:\Users\durle\anaconda3\Fauna_Detection\DataG2\images\train_patches
  val  : C:\Users\durle\anaconda3\Fauna_Detection\DataG2\images\val_patches


### Verificación CSVs de parches antes de la conversión

In [ ]:
import pandas as pd

print("=== Verificando CSVs de parches ===")

for csv_file in [TRAIN_PATCHES_CSV, VAL_PATCHES_CSV]:
    print("\nArchivo:", csv_file)

    # Verificar existencia
    if not csv_file.exists():
        print("ERROR: No encontrado:", csv_file)
        continue

    # Mostrar tamaño y primeras líneas
    df = pd.read_csv(csv_file)
    print("Filas:", len(df))
    print("Columnas:", list(df.columns))
    print(df.head())

=== Verificando CSVs de parches ===

Archivo: C:\Users\durle\anaconda3\Fauna_Detection\DataG2\train_patches.csv
Filas: 14058
Columnas: ['images', 'labels', 'base_images', 'x', 'y']
                                           images  labels  \
0  000113a692ba61cd55ea3acb9c2f9c41709710a1_0.JPG       6   
1  000113a692ba61cd55ea3acb9c2f9c41709710a1_0.JPG       6   
2  000113a692ba61cd55ea3acb9c2f9c41709710a1_1.JPG       6   
3  000113a692ba61cd55ea3acb9c2f9c41709710a1_1.JPG       6   
4  000113a692ba61cd55ea3acb9c2f9c41709710a1_1.JPG       6   

                                    base_images      x      y  
0  000113a692ba61cd55ea3acb9c2f9c41709710a1.JPG  375.0  236.5  
1  000113a692ba61cd55ea3acb9c2f9c41709710a1.JPG  430.0  462.0  
2  000113a692ba61cd55ea3acb9c2f9c41709710a1.JPG  228.5  294.0  
3  000113a692ba61cd55ea3acb9c2f9c41709710a1.JPG   23.0  236.5  
4  000113a692ba61cd55ea3acb9c2f9c41709710a1.JPG   78.0  462.0  

Archivo: C:\Users\durle\anaconda3\Fauna_Detection\DataG2\gt.csv
Fil

### 4.1) Conversión CSV --> YOLO .txt (aplicación)

In [ ]:
YOLO_CLASSES = ["buffalo","elephant","kob","topi","warthog","waterbuck"]  # 6 clases (índices 0..5)
N_CLASSES = len(YOLO_CLASSES)

def _crear_labels_yolo_desde_csv(csv_path: Path, images_dir: Path, labels_out_dir: Path,
                                 box_side_px: int = BOX_SIDE_PX):
    df = pd.read_csv(csv_path)
    # Columnas esperadas
    # - Para *patches*: images, labels, base_images, x, y
    # Validar nombres e intentar mapear
    cols = {c.lower(): c for c in df.columns}
    # Mapeos robustos
    col_img   = cols.get("images") or cols.get("image") or cols.get("file") or list(df.columns)[0]
    col_label = cols.get("labels") or cols.get("class") or cols.get("category")
    col_x     = cols.get("x")
    col_y     = cols.get("y")
    assert all([col_img, col_label, col_x, col_y]), f"CSV {csv_path.name} no contiene columnas esperadas. Tiene: {list(df.columns)}"

    # Ajuste por indexado 1..N si aplica
    delta = _inferir_indexado_cero_o_uno(df[col_label], N_CLASSES)

    # Agrupar por archivo de imagen
    grupos = defaultdict(list)
    for _, r in df.iterrows():
        try:
            cls = int(r[col_label]) + delta
        except:
            continue
        img_name = str(r[col_img])
        x = float(r[col_x]); y = float(r[col_y])
        grupos[img_name].append((cls, x, y))

    # Crear un caché de tamaños de imagen para normalizar
    size_cache = {}

    escritos = 0
    faltantes = 0

    for img_name, anotaciones in tqdm(grupos.items(), desc=f"Etiquetas YOLO: {csv_path.name}"):
        img_path = images_dir / img_name
        if not img_path.exists():
            # algunos parches podrían tener extensión distinta; intentar case-insensitive
            candidatos = list(images_dir.glob(img_name))
            if not candidatos:
                faltantes += 1
                continue
            img_path = candidatos[0]

        if img_path not in size_cache:
            im = cv2.imread(str(img_path))
            if im is None:
                faltantes += 1
                continue
            h, w = im.shape[:2]
            size_cache[img_path] = (w, h)
        else:
            w, h = size_cache[img_path]

        # Construir líneas YOLO
        yolo_lines = []
        for (cls, cx, cy) in anotaciones:
            bw = min(box_side_px, w)  # ancho caja en píxeles
            bh = min(box_side_px, h)  # alto caja en píxeles
            # Normalización YOLO
            x_c = max(0, min(w, cx)) / w
            y_c = max(0, min(h, cy)) / h
            w_n = bw / w
            h_n = bh / h
            # Recortar en bordes (evitar >1.0)
            x_c = max(0.0, min(1.0, x_c))
            y_c = max(0.0, min(1.0, y_c))
            w_n = max(1e-6, min(1.0, w_n))
            h_n = max(1e-6, min(1.0, h_n))
            yolo_lines.append(f"{int(cls)} {x_c:.6f} {y_c:.6f} {w_n:.6f} {h_n:.6f}")

        (labels_out_dir).mkdir(parents=True, exist_ok=True)
        txt_path = (labels_out_dir / img_path.with_suffix(".txt").name)
        with open(txt_path, "w") as f:
            f.write("\n".join(yolo_lines))
        escritos += 1

    print(f"Escritos: {escritos}  |  Imágenes en CSV sin archivo correspondiente: {faltantes}")

# Ejecutar conversión para train/val
_crear_labels_yolo_desde_csv(TRAIN_PATCHES_CSV, TRAIN_PATCHES_DIR, LABELS_ROOT/"train_patches")
_crear_labels_yolo_desde_csv(VAL_PATCHES_CSV,   VAL_PATCHES_DIR,   LABELS_ROOT/"val_patches")

Etiquetas YOLO: train_patches.csv: 100%|██████████| 7765/7765 [00:09<00:00, 824.69it/s] 


Escritos: 7765  |  Imágenes en CSV sin archivo correspondiente: 0


Etiquetas YOLO: gt.csv: 100%|██████████| 532/532 [00:00<00:00, 951.83it/s] 

Escritos: 532  |  Imágenes en CSV sin archivo correspondiente: 0


### Validar archivos .txt generados

In [ ]:
from pathlib import Path

train_txt = list((LABELS_ROOT/"train_patches").glob("*.txt"))
val_txt   = list((LABELS_ROOT/"val_patches").glob("*.txt"))

print("Archivos YOLO .txt generados:")
print("  Train:", len(train_txt))
print("  Val  :", len(val_txt))

print("\nEjemplo primeros 3 .txt en train:")
for t in train_txt[:5]:
    print("-", t.name)

Archivos YOLO .txt generados:
  Train: 7765
  Val  : 532

Ejemplo primeros 3 .txt en train:
- 000113a692ba61cd55ea3acb9c2f9c41709710a1_0.txt
- 000113a692ba61cd55ea3acb9c2f9c41709710a1_1.txt
- 000113a692ba61cd55ea3acb9c2f9c41709710a1_16.txt
- 000113a692ba61cd55ea3acb9c2f9c41709710a1_17.txt
- 002296711c356eca5bf02af0beaa0723ca2e5577_65.txt


#### Filtrar val_patches

In [ ]:
VAL_PATCHES_FILTERED = DATA_DIR / "images" / "val_patches_filtered"
VAL_PATCHES_FILTERED.mkdir(parents=True, exist_ok=True)

# Copiar solo imágenes presentes en gt.csv
df_val = pd.read_csv(VAL_PATCHES_CSV)
imgs_val = df_val['images'].unique()

from shutil import copy2
cop = 0

for img in imgs_val:
    src = VAL_PATCHES_DIR / img
    if src.exists():
        copy2(src, VAL_PATCHES_FILTERED / img)
        cop += 1

print(f"[INFO] val_patches filtrado: {cop} imágenes con anotación.")

[INFO] val_patches filtrado: 532 imágenes con anotación.


#### Copia las etiquetas en la carpeta esperada por YOLO

In [ ]:
# Crear etiquetas YOLO en labels/val_patches_filtered
VAL_LABELS_FILTERED = LABELS_ROOT / "val_patches_filtered"
VAL_LABELS_FILTERED.mkdir(parents=True, exist_ok=True)

# REGENERAR .txt SOLO para las imágenes filtradas
# Usar VAL_PATCHES_FILTERED como images_dir para que el generador
# escriba etiquetas únicamente de los archivos presentes (gt.csv se usa de filtro).
_crear_labels_yolo_desde_csv(
    VAL_PATCHES_CSV,          # gt.csv
    VAL_PATCHES_FILTERED,     # images_dir = SOLO las filtradas
    VAL_LABELS_FILTERED,      # destino correcto que YOLO buscará
    box_side_px=BOX_SIDE_PX
)

Etiquetas YOLO: gt.csv: 100%|██████████| 532/532 [00:00<00:00, 728.91it/s]

Escritos: 532  |  Imágenes en CSV sin archivo correspondiente: 0


In [ ]:
# verificación
val_imgs = sorted([p.name for p in _listar_imagenes(VAL_PATCHES_FILTERED)])
val_txts = sorted([p.name for p in (LABELS_ROOT/"val_patches_filtered").glob("*.txt")])

print("[CHECK] val imágenes :", len(val_imgs))
print("[CHECK] val etiquetas:", len(val_txts))

faltantes = [Path(n).with_suffix(".txt").name for n in val_imgs
             if not (LABELS_ROOT/"val_patches_filtered"/Path(n).with_suffix(".txt").name).exists()]
print("[CHECK] imágenes sin .txt:", len(faltantes))
if faltantes[:5]:
    print("  Ejemplos:", faltantes[:5])

[CHECK] val imágenes : 532
[CHECK] val etiquetas: 532
[CHECK] imágenes sin .txt: 0


### Conversión test a YOLO

In [ ]:
TEST_CSV = DATA_DIR / "test.csv"
TEST_LABELS_DIR = LABELS_ROOT / "test"
TEST_LABELS_DIR.mkdir(parents=True, exist_ok=True)

if TEST_CSV.exists():
    print("[INFO] test.csv encontrado --> convirtiendo a YOLO")
    _crear_labels_yolo_desde_csv(
        TEST_CSV,
        TEST_DIR,
        TEST_LABELS_DIR,
        box_side_px=BOX_SIDE_PX
    )
else:
    print("[INFO] No existe test.csv --> Solo se realizarán predicciones.")

[INFO] test.csv encontrado --> convirtiendo a YOLO


Etiquetas YOLO: test.csv: 100%|██████████| 258/258 [00:30<00:00,  8.52it/s]

Escritos: 258  |  Imágenes en CSV sin archivo correspondiente: 0


### 5) Definición del modelo YOLO + pérdidas (wrapper)

In [ ]:
from ultralytics import YOLO

class YOLOLossWrapper:
    def __init__(self, model):
        self.model = model
    def __getattr__(self, name):
        return getattr(self.model, name)

# Modelo base (colocar el .pt si es local o se descargará por nombre)
MODEL_PATH = BASE_DIR / "yolo11m.pt"
yolo_model = YOLOLossWrapper(YOLO(str(MODEL_PATH) if MODEL_PATH.exists() else "yolo11m.pt"))
print("YOLOv11m cargado")

YOLOv11m cargado


### 6) Configuración de entrenamiento, evaluador, métricas
(dataset.yaml apunta a parches existentes y labels generadas)

In [ ]:
from ultralytics.utils import LOGGER
import yaml

dataset_yaml = DATA_DIR / "dataset_yolo.yaml"

data_dict = {
    "path": str(DATA_DIR),
    "train": str(DATA_DIR /"images"/"train_patches"),
    "val":   str(DATA_DIR /"images"/"val_patches_filtered"),
    "names": {i:n for i,n in enumerate(YOLO_CLASSES)}
}

if TEST_CSV.exists():
    data_dict["test"] = str(TEST_DIR)

yaml.safe_dump(data_dict, open(dataset_yaml, "w"))

class YOLOMetrics:
    pass  # YOLO calcula métricas internamente

class YOLOEvaluator:
    def __init__(self, model, split, work_dir):
        self.model = model
        self.split = split
        self.work_dir = Path(work_dir); self.work_dir.mkdir(parents=True, exist_ok=True)

    def evaluate(self, returns="f1_score"):
        r = self.model.val(
            data=str(dataset_yaml),
            split=self.split,      # 'val' o 'test'
            imgsz=1024,
            conf=0.25,
            iou=0.6,
            device=0 if torch.cuda.is_available() else None,
        )
        # F1 aproximado por precisión/recuperación de bounding boxes
        p  = r.results_dict.get("metrics/precision(B)",0)
        rc = r.results_dict.get("metrics/recall(B)",0)
        f1 = 2*p*rc/(p+rc+1e-6)
        LOGGER.info(f"[{self.split}] F1 ≈ {f1:.4f}")
        return f1

class YOLOTrainer:
    def __init__(self, model, epochs, work_dir):
        self.model = model
        self.epochs = epochs
        self.work_dir = Path(work_dir); self.work_dir.mkdir(parents=True, exist_ok=True)

    def start(self):
        self.model.train(
            data=str(dataset_yaml),
            epochs=self.epochs,
            imgsz=1024,
            batch=8,
            device=0 if torch.cuda.is_available() else None,
            project=str(self.work_dir.parent),
            name=self.work_dir.name,
            save=True,
            save_period=1,  # GUARDAR CHECKPOINTS CADA ÉPOCA
            exist_ok=True
        )

### 6.1) Comprobar imágenes corruptas

In [ ]:
bad = 0; good = 0
for p in _listar_imagenes(DATA_DIR/"images"/"train_patches"):
    img = cv2.imread(str(p))
    if img is None: bad += 1
    else: good += 1
print("[train_patches] Buenas:", good, "Corruptas/No legibles:", bad)

bad = 0; good = 0
for p in _listar_imagenes(DATA_DIR/"images"/"val_patches_filtered"):
    img = cv2.imread(str(p))
    if img is None: bad += 1
    else: good += 1
print("[val_patches]   Buenas:", good, "Corruptas/No legibles:", bad)

[train_patches] Buenas: 7765 Corruptas/No legibles: 0
[val_patches]   Buenas: 532 Corruptas/No legibles: 0


### 7) Entrenamiento inicial (fase 1)

In [ ]:
work_dir_f1 = BASE_DIR/"runs_yolo"/"fase_1"
trainer = YOLOTrainer(
    model=yolo_model.model,
    epochs=100,
    work_dir=work_dir_f1
)
trainer.start()

New https://pypi.org/project/ultralytics/8.3.226 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.216  Python-3.10.18 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080 Laptop GPU, 16303MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\durle\anaconda3\Fauna_Detection\DataG2\dataset_yolo.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:\Users\durle\anaconda3\Fauna_Detection\y

2025/11/08 09:47:20 INFO mlflow.tracking.fluent: Experiment with name 'C:\Users\durle\anaconda3\Fauna_Detection\runs_yolo' does not exist. Creating a new experiment.


MLflow: logging run_id(54945cfc99c14596a549be9e4fd023bc) to runs\mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs\mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to C:\Users\durle\anaconda3\Fauna_Detection\runs_yolo\fase_1
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      1/100      9.91G       2.18      3.642      1.973          9       1024: 100% ━━━━━━━━━━━━ 971/971 3.2it/s 5:04<0.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 34/34 3.6it/s 9.4s0.2s
                   all        532        977      0.394      0.251       0.22      0.094

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      2/100      10.1G      1.929      2.085      1.776          4       1024: 100% ━━━━━━━━━━━━ 971/971 3.3it/s 

### Verificar fuga por base_images

In [ ]:
import pandas as pd

df_train_p = pd.read_csv(DATA_DIR/"train_patches.csv")
df_val_p   = pd.read_csv(DATA_DIR/"gt.csv")

tr_bases = set(df_train_p["base_images"].astype(str).unique())
va_bases = set(df_val_p["base_images"].astype(str).unique())
overlap  = tr_bases & va_bases

print("Bases en train:", len(tr_bases))
print("Bases en val_patches:", len(va_bases))
print("Bases solapadas:", len(overlap))
if len(overlap)>0:
    print("Ejemplos:", list(sorted(overlap))[:10])
else:
    print("Sin fuga por base_images entre train y val_patches")

Bases en train: 928
Bases en val_patches: 111
Bases solapadas: 0
Sin fuga por base_images entre train y val_patches


## 8) Evaluación con validación

In [ ]:
val_eval  = YOLOEvaluator(yolo_model.model, "val",  work_dir_f1)
val_f1    = val_eval.evaluate()
print("F1 val:", val_f1)

Ultralytics 8.3.216  Python-3.10.18 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080 Laptop GPU, 16303MiB)
YOLO11m summary (fused): 125 layers, 20,034,658 parameters, 0 gradients, 67.7 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 92.137.5 MB/s, size: 31.8 KB)
val: Scanning C:\Users\durle\anaconda3\Fauna_Detection\DataG2\labels\val_patches_filtered.cache... 532 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 532/532 530.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 34/34 2.8it/s 12.3s0.3s
                   all        532        977      0.796      0.682       0.76      0.465
               buffalo        209        368       0.88       0.81      0.861      0.513
              elephant         53        102      0.853      0.735      0.819      0.405
                   kob        105        161      0.709      0.888      0.816      0.571
                  topi         21         43      0.744       